In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import pickle
#import pubchempy as pcp
#from chembl_webresource_client.new_client import new_client
from azureml.core import Workspace, Dataset, Datastore
import copy


In [20]:
# read the data lobe
subscription_id = 'b88e2867-6b66-4723-88c1-25f2d77b0394'
resource_group = 'az-srotasengine-dev-uks-001'
workspace_name = 'srotasengine'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "workspaceblobstore")
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'UI/2024-11-05_052811_UTC/simulacrum/Data'))


In [27]:
# Assuming the previous code to list CSV files is already present
datastore_path = [(datastore, 'UI/2024-11-05_052811_UTC/simulacrum/')]
dataset = Dataset.File.from_files(path=datastore_path)
file_paths = dataset.to_path()
csv_files = [file for file in file_paths if file.endswith('.csv')]

# Read only the first CSV file
first_csv_path = '/Data/sim_sact_regimen.csv' #csv_files[0]
local_path = dataset.download(target_path='.', overwrite=True)
first_csv_df = pd.read_csv(local_path[0])
# Check if there are any CSV files and read the first one
# if csv_files:
#     first_csv_path = csv_files[0]
    
#     # Use dataset.download to retrieve the file
#     local_path = dataset.download(target_path='.', overwrite=True)
    
#     # Read the first CSV file using pandas
#     first_csv_df = pd.read_csv(local_path[0])  # local_path[0] refers to the downloaded first CSV file
    
#     # Display the first few rows to confirm it's loaded
#     print(first_csv_df.head())
# else:
#     print("No CSV files found.")

{'infer_column_types': 'False', 'activity': 'to_path'}
{'infer_column_types': 'False', 'activity': 'to_path', 'activityApp': 'FileDataset'}
{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 556: invalid start byte

# process the tumour data

In [39]:
data_dir = '/home/azureuser/cloudfiles/code/Users/Omid.Bazgir/data/'

patient_pd = pd.read_csv(data_dir + 'sim_av_patient.csv')
tumour_pd = pd.read_csv(data_dir + 'sim_av_tumour.csv')
regimen_pd = pd.read_csv(data_dir + 'sim_sact_regimen.csv')
outcomes_pd = pd.read_csv(data_dir + 'sim_sact_outcome.csv')
cancer_types = pd.read_json(data_dir + 'cancer_types.json')
drug_pd = pd.read_csv(data_dir + 'sim_sact_drug_detail.csv')

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: DtypeWarning: Columns (14,15,16,17,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
# can you make json file from the below code
Ethnicity = {}
Ethnicity['White'] = ['0','A','B','C','CA','CH','CP']
Ethnicity['Mixed'] = ['D','E','F','G']
Ethnicity['SouthAsian'] = ['H','J','K','L']
Ethnicity['Black'] = ['M','N','P']
Ethnicity['Asian'] = ['R']
Ethnicity['Other'] = ['S','Z','X']
# save the dictionary as a json file
import json
with open('ethnicity.json', 'w') as json_file:
    json.dump(Ethnicity, json_file, indent=4)


In [6]:
# read ethnicity data from the json file
with open(data_dir + 'ethnicity.json', 'r') as json_file:
    Ethnicity = json.load(json_file)


In [10]:
# apply the mapping from the Ethnicity dictionary to the patient_pd dataframe ETHNICITY column
# Create a reverse mapping dictionary
reverse_mapping = {code: ethnicity for ethnicity, codes in Ethnicity.items() for code in codes}

# Apply the reverse mapping to the ETHNICITY column
patient_pd['ETHNICITY'] = patient_pd['ETHNICITY'].map(reverse_mapping)


In [20]:
# read the patient data from the dataset pickle file
patient_data = pd.read_pickle(data_dir + 'patient_data.pkl')
merged_demographic = patient_data['demographic']
# add ethnicity data from patient_pd to the merged_demographic dataframe for each PATIENTID
merged_demographic = merged_demographic.merge(patient_pd[['PATIENTID', 'ETHNICITY']], on='PATIENTID', how='left')
patient_data['demographic'] = merged_demographic
# save the patient_data as a pickle file
patient_data_path = data_dir + 'patient_data.pkl'
with open(patient_data_path, 'wb') as file:
    pickle.dump(patient_data, file)

In [17]:


# Merge the dataframes on PATIENTID


In [18]:
merged_demographic

,GENDER,PATIENTID,Cancer Type,AGE,ENCORE_PATIENT_ID,HEIGHT_AT_START_OF_REGIMEN,WEIGHT_AT_START_OF_REGIMEN,ETHNICITY
0,1,10300637,Breast Cancer,87,10300637,1.76,79.50,White
1,2,10300837,Breast Cancer,64,10300837,1.67,78.75,White
2,1,10300848,Breast Cancer,94,10300848,1.67,107.10,White
3,1,10301090,Breast Cancer,69,10301090,1.71,90.00,NaN
4,1,10301209,Breast Cancer,90,10301209,1.62,68.30,White
...,...,...,...,...,...,...,...,...
131307,2,160016392,Colorectal Cancer,81,160016392,1.65,51.20,White
131308,2,170006397,Colorectal Cancer,69,170006397,1.70,83.00,SouthAsian
131309,2,190004163,Colorectal Cancer,88,190004163,1.61,69.50,White
131310,2,200006421,Colorectal Cancer,56,200006421,1.52,70.40,White


## processing outcome dataset to create weak label for supervised learning

In [3]:
# strong labels
# save the outcome_label_pd as a pickle file
label_pickle = {}

# create a dictionary of labels based on the each regimen modification type
label_dict = {}
# dose reduction is an indication of toxicity and safety issues
dose_reduction = {'Y': 0, 'N': None} 
label_dict['REGIMEN_MOD_DOSE_REDUCTION'] = dose_reduction
# early stop is an indication of success of the treatment
early_stop = {'Y': 1, 'N': None} 
label_dict['REGIMEN_MOD_STOPPED_EARLY'] = early_stop

# time delay is an indication of safety and efficacy
time_delay = {'Y': 0, 'N': None}
label_dict['REGIMEN_MOD_TIME_DELAY'] = time_delay
# regimen outcome summary is an indication of the failure of the treatment according to the data specifications
outcome_summary_dict = {'0': None,'1':0, '2':0, '3':0, '4':0, '5': None}
label_dict['REGIMEN_OUTCOME_SUMMARY'] = outcome_summary_dict

# make strong labels for the outcomes
strong_outcome_label_pd = copy.copy(outcomes_pd)
for key in label_dict.keys():
    strong_outcome_label_pd[key] = strong_outcome_label_pd[key].map(label_dict[key])

label_pickle['strong_label'] = strong_outcome_label_pd

In [20]:
print("STRONG LABELS")
strong_outcome_label_pd['Label'] = strong_outcome_label_pd['REGIMEN_OUTCOME_SUMMARY']
print(strong_outcome_label_pd['Label'].value_counts())
# if the label is NaN, and REGIMEN_MOD_DOSE_REDUCTION is 0, then the label is 0
# find where the label is NaN and REGIMEN_MOD_DOSE_REDUCTION is 0
index = strong_outcome_label_pd['Label'].isna() & (strong_outcome_label_pd['REGIMEN_MOD_DOSE_REDUCTION'] == 0)
strong_outcome_label_pd.loc[index, 'Label'] = 0
print(strong_outcome_label_pd['Label'].value_counts())
# if the label is NaN, and REGIMEN_MOD_TIME_DELAY is 0, then the label is 0
# find where the label is NaN and REGIMEN_MOD_TIME_DELAY is 0
index = strong_outcome_label_pd['Label'].isna() & (strong_outcome_label_pd['REGIMEN_MOD_TIME_DELAY'] == 0)
strong_outcome_label_pd.loc[index, 'Label'] = 0
print(strong_outcome_label_pd['Label'].value_counts())
# if the label is NaN, and REGIMEN_MOD_STOPPED_EARLY is 1, then the label is 1
# find where the label is NaN and REGIMEN_MOD_STOPPED_EARLY is 1
index = strong_outcome_label_pd['Label'].isna() & (strong_outcome_label_pd['REGIMEN_MOD_STOPPED_EARLY'] == 1)
strong_outcome_label_pd.loc[index, 'Label'] = 1
print(strong_outcome_label_pd['Label'].value_counts())
label_pickle['strong_label'] = strong_outcome_label_pd

STRONG LABELS
0.0    33022
Name: Label, dtype: int64
0.0    192702
Name: Label, dtype: int64
0.0    262372
Name: Label, dtype: int64
0.0    262372
1.0     39213
Name: Label, dtype: int64


In [4]:
# make weak labels for the outcomes
# dose reduction is an indication of toxicity and safety issues
dose_reduction = {'Y': 0, 'N': 1} 
label_dict['REGIMEN_MOD_DOSE_REDUCTION'] = dose_reduction
# early stop is an indication of success of the treatment
early_stop = {'Y': 1, 'N': 0} 
label_dict['REGIMEN_MOD_STOPPED_EARLY'] = early_stop

# time delay is an indication of safety and efficacy
time_delay = {'Y': 0, 'N': 1}
label_dict['REGIMEN_MOD_TIME_DELAY'] = time_delay
# regimen outcome summary is an indication of the failure of the treatment according to the data specifications
outcome_summary_dict = {'0': 1,'1':0, '2':0, '3':0, '4':0, '5': 1}
label_dict['REGIMEN_OUTCOME_SUMMARY'] = outcome_summary_dict

# make strong labels for the outcomes
weak_outcome_label_pd = copy.copy(outcomes_pd)
for key in label_dict.keys():
    weak_outcome_label_pd[key] = weak_outcome_label_pd[key].map(label_dict[key])

label_pickle['weak_label'] = weak_outcome_label_pd


In [27]:
print("WEAK LABELS")
weak_outcome_label_pd['Label'] = weak_outcome_label_pd['REGIMEN_OUTCOME_SUMMARY']
print(weak_outcome_label_pd['Label'].value_counts())
# if the label is NaN, and REGIMEN_MOD_STOPPED_EARLY is 1, then the label is 1
# find where the label is NaN and REGIMEN_MOD_STOPPED_EARLY is 1
index = weak_outcome_label_pd['Label'].isna() & (weak_outcome_label_pd['REGIMEN_MOD_STOPPED_EARLY'] == 1)
weak_outcome_label_pd.loc[index, 'Label'] = 1
print(weak_outcome_label_pd['Label'].value_counts())
# if the label is NaN, and REGIMEN_MOD_DOSE_REDUCTION is 0, then the label is 0
# find where the label is NaN and REGIMEN_MOD_DOSE_REDUCTION is 0
index = weak_outcome_label_pd['Label'].isna() & (weak_outcome_label_pd['REGIMEN_MOD_DOSE_REDUCTION'] == 0)
weak_outcome_label_pd.loc[index, 'Label'] = 0
print(weak_outcome_label_pd['Label'].value_counts())
# if the label is NaN, and REGIMEN_MOD_TIME_DELAY is 0, then the label is 0
# find where the label is NaN and REGIMEN_MOD_TIME_DELAY is 0
index = weak_outcome_label_pd['Label'].isna() & (weak_outcome_label_pd['REGIMEN_MOD_TIME_DELAY'] == 0)
weak_outcome_label_pd.loc[index, 'Label'] = 0
print(weak_outcome_label_pd['Label'].value_counts())
weak_outcome_label_pd['Label'].fillna(0, inplace=True)
print(weak_outcome_label_pd['Label'].value_counts())
label_pickle['weak_label'] = weak_outcome_label_pd



WEAK LABELS
1.0    70508
0.0    33022
Name: Label, dtype: int64
1.0    126539
0.0     33022
Name: Label, dtype: int64
0.0    163540
1.0    126539
Name: Label, dtype: int64
0.0    213230
1.0    126539
Name: Label, dtype: int64
0.0    657596
1.0    126539
Name: Label, dtype: int64


In [47]:
# save the label_pickle as a pickle file
with open(data_dir + 'label_pickle.pkl', 'wb') as f:
    pickle.dump(label_pickle, f)

In [3]:
# post process the genomics data
# read the genomics data as the pickle file
with open(data_dir + 'patient_data.pkl', 'rb') as f:
    patient_data = pickle.load(f)

In [11]:
columns = patient_data['genomics'].columns.tolist()
columns.remove('PATIENTID')
columns.remove('TUMOURID')
gene_pd_merged = patient_data['genomics']

In [32]:
patient_id = gene_pd_merged['PATIENTID'].unique()
gene_aggr_ls = []
for ptn_id in patient_id:
    gene_ptn = gene_pd_merged[gene_pd_merged['PATIENTID'] == ptn_id]
    # in each column set the value to the maximum of the gene expression
    gene_ptn = gene_ptn.agg('max')
    gene_ptn = pd.DataFrame(gene_ptn).T
    gene_aggr_ls.append(gene_ptn)
gene_aggr_ls = pd.concat(gene_aggr_ls)

,ROS1_EXP,NRAS_EXP,CCND2_EXP,CCND1_EXP,CALR_EXP,IDH1_EXP,EGFR_EXP,CD274 (PD-L1)_EXP,ERBB2 (HER2 / NEU)_EXP,BRAF_EXP,...,MPL_DNA,SDHB_DNA,JAK2_DNA,MALT1_DNA,BCL2_DNA,BCL6_DNA,EZH2_DNA,CIC_DNA,HIST1H3B_DNA,IGH_DNA
0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,0,-1,-1,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,0,3,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,0,-1,-1,-1,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,ROS1_EXP,NRAS_EXP,CCND2_EXP,CCND1_EXP,CALR_EXP,IDH1_EXP,EGFR_EXP,CD274 (PD-L1)_EXP,ERBB2 (HER2 / NEU)_EXP,BRAF_EXP,...,MPL_DNA,SDHB_DNA,JAK2_DNA,MALT1_DNA,BCL2_DNA,BCL6_DNA,EZH2_DNA,CIC_DNA,HIST1H3B_DNA,IGH_DNA
0,-1,0,-1,-1,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [7]:
unique_values = []
for cl in columns:
    uni_cl_val = patient_data['genomics'][cl].unique()
    for val in uni_cl_val:
        if val not in unique_values:
            unique_values.append(val)
    

In [8]:
# create a dictionary for unique_values
unique_values_dict = {}
for i, val in enumerate(unique_values):
    unique_values_dict[val] = i
unique_values_dict[-1]= -1

In [9]:
# for every column, map the unique values to integers
for cl in columns:
    patient_data['genomics'][cl] = patient_data['genomics'][cl].map(unique_values_dict)

In [10]:
patient_data['genomics']['TUMOURID']

43562     10000151
69339     10000151
43926     10000183
43253     10000518
115528    10000699
            ...   
125870    11999966
148992    11999966
172365    11999966
194184    11999966
207301    11999966
Name: TUMOURID, Length: 66736, dtype: int64